Author : Remus !!
Date : 2015.09.07

Exercise:
- Calculate surivival rate for titianic scenario from Kaggle
- Use Perceptron Learning Algorithm (PLA) from sklearn kit

Assumption:
- This exercise does not include the ordinary "data exploration" step, as this step has been carried out by other exercise.

Steps:
0. Import project packages
1. Data Cleanup
2. Fit the model
3. Predict the test data

In [3]:
import pandas as pd
import numpy as np
import csv as csv
#from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron

1. Data Cleanup
- Convert all strings to interger classifers
- Fill in the missing values of the data and make it complete
- Do for both "training" and "test" dataset

In [4]:
# DATA : Training data
train_df = pd.read_csv('train.csv', header=0)        # Load the train file into a dataframe

In [5]:
# SEX => GENDER : female = 0, Male = 1
train_df['Gender'] = train_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# ==============================================================================
# EMBARKED : Embarked from 'C', 'Q', 'S'
# Note this is not ideal: in translating categories to numbers, Port "2" is not 2 times greater than Port "1", etc.

# All missing Embarked -> just make them embark from most common place
if len(train_df.Embarked[ train_df.Embarked.isnull() ]) > 0:
    train_df.Embarked[ train_df.Embarked.isnull() ] = train_df.Embarked.dropna().mode().values

Ports = list(enumerate(np.unique(train_df['Embarked'])))    # determine all values of Embarked,
Ports_dict = { name : i for i, name in Ports }              # set up a dictionary in the form  Ports : index
train_df.Embarked = train_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)     # Convert all Embark strings to int

# ==============================================================================
# AGE : All the ages with no data -> make the median of all Ages
median_age = train_df['Age'].dropna().median()
if len(train_df.Age[ train_df.Age.isnull() ]) > 0:
    train_df.loc[ (train_df.Age.isnull()), 'Age'] = median_age

# ==============================================================================
# All the missing Fares -> assume median of their respective class
if len(train_df.Fare[ train_df.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = train_df[ train_df.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        train_df.loc[ (train_df.Fare.isnull()) & (train_df.Pclass == f+1 ), 'Fare'] = median_fare[f]    

# ==============================================================================
# Remove the Name column, Cabin, Ticket, and Sex (since I copied and filled it to Gender)
train_df = train_df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'PassengerId'], axis=1)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
# DATA : TEST DATA
test_df = pd.read_csv('test.csv', header=0)        # Load the test file into a dataframe

In [7]:
# SEX => GENDER : female = 0, Male = 1
test_df['Gender'] = test_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# ==============================================================================
# EMBARKED : Embarked from 'C', 'Q', 'S'
# All missing Embarked -> just make them embark from most common place
if len(test_df.Embarked[ test_df.Embarked.isnull() ]) > 0:
    test_df.Embarked[ test_df.Embarked.isnull() ] = test_df.Embarked.dropna().mode().values
# Again convert all Embarked strings to int
test_df.Embarked = test_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)

# ==============================================================================
# AGE : All the ages with no data -> make the median of all Ages
median_age = test_df['Age'].dropna().median()
if len(test_df.Age[ test_df.Age.isnull() ]) > 0:
    test_df.loc[ (test_df.Age.isnull()), 'Age'] = median_age

# ==============================================================================
# All the missing Fares -> assume median of their respective class
if len(test_df.Fare[ test_df.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = test_df[ test_df.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        test_df.loc[ (test_df.Fare.isnull()) & (test_df.Pclass == f+1 ), 'Fare'] = median_fare[f]

# ==============================================================================
# Collect the test data's PassengerIds before dropping it
ids = test_df['PassengerId'].values
# Remove the Name column, Cabin, Ticket, and Sex (since I copied and filled it to Gender)
test_df = test_df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'PassengerId'], axis=1) 

2. Fit the model
- Convert the data from dataframe back to numpy array
- Fit the model : Perceptron Learning Algorithm (PLA) from sklearn kit

In [8]:
# The data is now ready to go. So lets fit to the train, then predict to the test!
# Convert back to a numpy array
train_data = train_df.values
test_data = test_df.values

In [9]:
print 'Training...'
#forest = RandomForestClassifier(n_estimators=100)
#forest = forest.fit( train_data[0::,1::], train_data[0::,0] )

pla = Perceptron(n_iter=50)
pla = pla.fit( train_data[0::,1::], train_data[0::,0] )

Training...


3. Predict the TEST dataset
- Make prediction on the "test.csv" dataset
- Output prediction outcomes to a new file "titanic_pla.csv"

In [10]:
print 'Predicting...'
#output = forest.predict(test_data).astype(int)
output = pla.predict(test_data).astype(int)

Predicting...


In [11]:
predictions_file = open("titanic_pla.csv", "wb")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["PassengerId","Survived"])
open_file_object.writerows(zip(ids, output))
predictions_file.close()
print 'Done.'

Done.
